# Code for training the holistic model

## Import Libraries 


In [9]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import os
FOLDER_NAME = '../dataset'

CLASSES = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [10]:
# Create label map
label_map = {}
for (root, folders, files) in os.walk(FOLDER_NAME):
    for foldername in folders:
        if foldername in CLASSES:
            label_map[foldername] = CLASSES.index(foldername)

print(label_map)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [11]:
sequence, label = [], []
target_length = 14
for (root, folders, files) in os.walk(FOLDER_NAME):
    for filename in files:
        file_path = os.path.join(os.path.relpath(
            root, FOLDER_NAME), filename)
        if (filename.endswith('.npy') and os.path.split(file_path)[0] in CLASSES):
            res = np.load(f'{FOLDER_NAME}/{file_path}')
            for _ in range(target_length-res.shape[0]):
                res = np.vstack((res, res[-1, :]))
            sequence.append(np.array(res))
            label.append(label_map[os.path.basename(root[-1])])

print(np.array(sequence).shape)
print(np.array(label).shape)

(552, 14, 1662)
(552,)


In [21]:
# FOR DATA DUPLICATION
sequence = np.concatenate([sequence, sequence, sequence, ], axis=0)
label = np.concatenate([label, label, label], axis=0)

print(np.array(sequence).shape)
print(np.array(label).shape)


(1656, 14, 1662)
(1656,)


In [61]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sequence), tf.keras.utils.to_categorical(
    np.array(label).astype(int), num_classes=np.array(CLASSES).shape[0], dtype='float32'), test_size=0.1)

print(X_train.shape, X_test.shape)


(1490, 14, 1662) (166, 14, 1662)


In [23]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard


In [62]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)


In [63]:
model = Sequential()
model.add(LSTM(64, return_sequences=True,
          activation='tanh', input_shape=(14, 1662)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(np.array(CLASSES).shape[0], activation='softmax'))


In [64]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


In [65]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [66]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])


Epoch 1/200
47/47 [==============================] - 4s 16ms/step - loss: 3.2141 - categorical_accuracy: 0.0718
Epoch 2/200
47/47 [==============================] - 1s 13ms/step - loss: 3.0844 - categorical_accuracy: 0.0537
Epoch 3/200
47/47 [==============================] - 1s 13ms/step - loss: 2.7582 - categorical_accuracy: 0.1094
Epoch 4/200
47/47 [==============================] - 1s 13ms/step - loss: 2.5457 - categorical_accuracy: 0.1275
Epoch 5/200
47/47 [==============================] - 1s 12ms/step - loss: 2.5051 - categorical_accuracy: 0.1322
Epoch 6/200
47/47 [==============================] - 1s 13ms/step - loss: 2.4989 - categorical_accuracy: 0.1128
Epoch 7/200
47/47 [==============================] - 1s 13ms/step - loss: 2.7344 - categorical_accuracy: 0.1074
Epoch 8/200
47/47 [==============================] - 1s 14ms/step - loss: 2.5022 - categorical_accuracy: 0.1215
Epoch 9/200
47/47 [==============================] - 1s 13ms/step - loss: 2.4916 - categorical_accuracy:

In [67]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 14, 64)            442112    
                                                                 
 lstm_7 (LSTM)               (None, 14, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 26)                858       
                                                                 
Total params: 597,434
Trainable params: 597,434
Non-tr

In [68]:
res = model.predict(X_test)

6/6 [==============================] - 1s 19ms/step


In [69]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Assume X_test and y_test are your test dataset
# Make predictions
y_pred = model.predict(X_test)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)  # if y_test is one-hot encoded

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred_classes)

# Calculate precision, recall, and F1 score for each class
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Detailed classification report
report = classification_report(y_true, y_pred_classes)
print(report)


6/6 [==============================] - 0s 19ms/step
Accuracy: 0.9156626506024096
Precision: 0.933082329317269
Recall: 0.9156626506024096
F1 Score: 0.9102202000316194
              precision    recall  f1-score   support

           0       1.00      0.64      0.78        11
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         3
           4       1.00      0.91      0.95        11
           5       1.00      1.00      1.00        14
           6       1.00      1.00      1.00         6
           7       0.73      1.00      0.85        11
           8       0.75      1.00      0.86         9
           9       1.00      1.00      1.00         4
          10       1.00      1.00      1.00         7
          11       0.92      1.00      0.96        11
          12       1.00      1.00      1.00         9
          13       1.00      1.00      1.00         8
          14       1.00

In [75]:
actions = np.array(CLASSES)
testing = 17
print(actions[np.argmax(res[testing])], actions[np.argmax(y_test[testing])])

f f


In [76]:
model.save('action.h5')

In [77]:
model.load_weights('action.h5')

In [78]:
yhat = model.predict(X_test)

6/6 [==============================] - 0s 20ms/step


In [79]:
ytrue = np.argmax(y_test, axis=1).tolist()


In [80]:
yhat = np.argmax(yhat, axis=1).tolist()

In [81]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
multilabel_confusion_matrix(ytrue, yhat)

array([[[155,   0],
        [  4,   7]],

       [[160,   0],
        [  0,   6]],

       [[164,   0],
        [  0,   2]],

       [[163,   0],
        [  0,   3]],

       [[155,   0],
        [  1,  10]],

       [[152,   0],
        [  0,  14]],

       [[160,   0],
        [  0,   6]],

       [[151,   4],
        [  0,  11]],

       [[154,   3],
        [  0,   9]],

       [[162,   0],
        [  0,   4]],

       [[159,   0],
        [  0,   7]],

       [[154,   1],
        [  0,  11]],

       [[157,   0],
        [  0,   9]],

       [[158,   0],
        [  0,   8]],

       [[161,   0],
        [  3,   2]],

       [[158,   3],
        [  0,   5]],

       [[160,   0],
        [  1,   5]],

       [[157,   2],
        [  3,   4]],

       [[163,   0],
        [  2,   1]],

       [[161,   1],
        [  0,   4]],

       [[164,   0],
        [  0,   2]],

       [[157,   0],
        [  0,   9]],

       [[160,   0],
        [  0,   6]],

       [[161,   0],
        [  0, 

In [82]:
accuracy_score(ytrue, yhat)

0.9156626506024096